In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import datetime

con = psycopg2.connect(dbname = dbname, user = sqluser, password = password, host = host, port = port)

In [ ]:
query = """
select subject_id,hadm_id,admittime from mimiciii.admissions order by subject_id, admittime
"""
df = pd.read_sql_query(query, con)

In [ ]:
# Diagnosis Code(group by elixhauser)
df_icd = pd.read_csv('elixhauser.csv')
df_icd = df_icd.drop(df_icd.columns[[0]], axis=1)
df_icd = df_icd.fillna(0)
df_icd = df_icd.set_index('hadm_id')
df_icd = df_icd.astype(np.int64)

data = df_icd.join(df, how = 'left')
data = data.reset_index()
data = data.set_index(['hadm_id','subject_id','admittime'])

In [ ]:
data['RENAL'] = np.where(data['renal_failure'] == 1, '1', '0')
data['CARDIOVASCULAR'] = np.where((data['congestive_heart_failure'] == 1 or
                                   data['cardiac_arrhythmias'] == 1 or
                                   data['valvular_disease'] == 1 or
                                   data['pulmonary_circulation'] == 1 or
                                   data['peripheral_vascular'] == 1
                                  ), 
                                  '1', '0')
data['RESPIRATORY'] = np.where(data['chronic_pulmonary'] == 1, '1', '0')
data['HEPATIC'] = np.where(data['liver_disease'] == 1, '1', '0')
data['CANCER'] = np.where((data['lymphoma'] == 1 or
                                   data['metastatic_cancer'] == 1 or
                                   data['solid_tumor'] == 1
                                  ), 
                                  '1', '0')
data['DIABETES'] = np.where((data['diabetes_uncomplicated'] == 1 or
                                   data['diabetes_complicated'] == 1 
                                  ), 
                                  '1', '0')
data['RENAL'] = np.where(data['renal_failure'] == 1, '1', '0')
data['MULTIPLE COMORBIDITIES'] =  np.where(data.sum(axis = 1) >= 2, '1', '0') 

In [ ]:
data.to_csv('current_comorbidities.csv')